# Data Preprocessing

Il dataset, prima di tutto, va opportunamente elaborato per occuparsi di eventuali standardizzazioni e feature da eliminare. Inoltre dovremo separare i dati per ottenere un training e un test set.
Gli step che seguiremo saranno i seguenti:
1. Caricare i dati
2. Eliminare le features ridondanti o inutili
3. Standardizzare i dati
4. Dividere il dataset in training e test set
5. Creazione di tensori e dataloader

### Definizione variabili

Per velocizzare il processo di prova di nuovi modelli, vengono dichiarate qui alcune `variabili` che verranno utilizzate successivamente nel codice.

In [ ]:
# Lunghezza percentuale del dataset di allenamento, il test è definito di conseguenza
len_percentage_train = 80 / 100

classes = ("Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral")

### Librerie

Allo stesso modo si inseriscono qui tutte le `librerie` usate per il progetto

In [ ]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import albumentations as A  # Questa libreria è servità per la data augmentation
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sn
IN_COLAB = 'google.colab' in sys.modules

### Caricamento dei dati

Tramite `pandas` e il dataset fornito in .csv, carichiamo i dati all'interno di un dataset.

La variabile `IN_COLAB` permette al sistema di distinguere se il codice è eseguito in locale o all'interno di _Google Colab_. 

In [ ]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    fer2013 = pd.read_csv('/content/drive/My Drive//Università//Deep Learning//emotion-cnn//files//fer2013.csv')
else:
    fer2013 = pd.read_csv('files/fer2013.csv')
fer2013

### Eliminazione della colonna "Usage"

All'interno del dataset è presente una colonna `Usage`, che è stata utilizzata durante la challenge da cui questo dataset è stato preso. Nel nostro caso non è utile, quindi la eliminiamo, utilizzando il metodo `.drop`.

In [ ]:
print(list(fer2013.columns))
fer2013 = fer2013.drop(['Usage'], axis=1)
print(list(fer2013.columns))

### Da stringa di pixels a `np.array`
In questo passaggio elaboriamo il campo `pixel` del dataset. Di default questo è una stringa di numeri separati da uno spazio.

Per trasformare questa stringa in un array di `numpy`:
1. Separiamo tutti i numeri con `.split(' ')`, che vengono subito inseriti in un array di `numpy`
2. L'array viene trasformato in una matrice 48x48: a questo punto i numero non verranno più considerati come stringhe, ma come `float32`
3. Tutti i valori vengono divisi per 255, in modo da avere solo numeri compresi da 0 e 1.

In [ ]:
fer2013.pixels = fer2013.pixels.apply(lambda x: np.array(x.split(' ')).reshape(1, 48, 48).astype('float32')/255)

### Salvataggio del dataset normalizzato
Tramite un oggetto `pickle`, salviamo il dataset processato. In questo modo non dovremo rieseguire i blocchi di codici precedenti, che sono computazionalmente pesanti.

In [ ]:
if IN_COLAB:
    fer2013.to_pickle("/content/drive/My Drive/Università/Deep Learning/emotion-cnn/files/fer_norm.pkl")
else:
    fer2013.to_pickle("files/fer_norm.pkl")

### Caricamento del dataset normalizzato

In [ ]:
if IN_COLAB:
    fer2013 = pd.read_pickle("/content/drive/My Drive/Università/Deep Learning/emotion-cnn/files/fer_norm.pkl")
else:
    fer2013 = pd.read_pickle("files/fer_norm.pkl")

fer2013

### Divisione del dataset in training e test set

La seguente divisione produrrà due dataset:
1. Il training set, rapprensentatnte l'80% del dataset iniziale
2. Il test set set, rappresentatnte del 20% del dataset iniziale

In [ ]:
train_set, test_set = np.split(fer2013, [ int(len_percentage_train * len(fer2013)) ] )
test_set.reset_index(inplace=True, drop=True) # Resettiamo gli index per semplificare le operazioni successive

print("Lunghezza percentuale di:")
print("train_set: {:.0%}:".format( len(train_set) / len(fer2013) ))
print("test_set: {:.0%}".format( len(test_set) / len(fer2013) ))

## Data augmentation
Dopo svariati test, abbiamo constatato che ci fosse bisogno di aumentare il volume di dati, in quanto, come visibile nel blocco di codice successivo, il numero di dati disponibile per emozione non è lo stesso per tutte.

Abbiamo provato sia a generare le immagini direttamente durante il training, dentro le epoche, sia prima del training, portanto tutte le classi ad avere lo stesso numero di dati. 

Nel nostro caso, il metodo più efficace si è rivelato quello di generare le immagini **prima** del training. Come livello comune abbiamo scelto quello della classe `Happy`, che contava più immagini di tutte.

In [ ]:
# Printiamo il numero di dati per ogni classe, salavando il nuero di dati più grande
max_elements = 0

for i, emote in enumerate(classes):
    print(emote, "-->", len(train_set[train_set.emotion == i]))

    if len(train_set[train_set.emotion == i]) > max_elements:
        max_elements = len(train_set[train_set.emotion == i])

In [ ]:
transform_train = A.Compose([
    A.HorizontalFlip(p=0.5), # Flip dell'immagine sull'asse y (immagine specchiata), con probabilità del 50% 
    A.RandomBrightnessContrast(p=0.3), # Randomizzazione della luminosità e del contrasto dell'immagine, con probabilità del 30%
    A.Rotate(limit = 30) # L'immagine viene ruotata in un range che va da un angolo di -30 a 30 gradi
])

def generate_data(elements_to_generate, transform_seq, emotion, images):
    i = 0
    result = []
    while i < elements_to_generate:
        for image in images:
            if i >= elements_to_generate:
                break
            
            transformed = transform_train(image=image)
            transformed_image = transformed["image"]
            em_img = emotion,transformed_image
            result.append(em_img)
            i+=1
    
    return result

In [ ]:
new_data = []
for idx, emotion in enumerate(classes):
    train_set = pd.concat([
        train_set,
        pd.DataFrame(
            generate_data(max_elements - len(train_set[train_set.emotion == idx]), transform_train, idx, train_set[train_set.emotion == idx].pixels),
            columns=["emotion", "pixels"]
        )
    ])

train_set = train_set.reset_index(drop=True)

### Analisi finale del dataset
A questo punto la quantità di dati di train sono tutti allo stesso livello:

In [ ]:
freq = [0] * 7
for image in train_set.emotion:
    freq[image] += 1

print("Train:", freq)

freq = [0] * 7
for image in test_set.emotion:
    freq[image] += 1

print("Test:", freq)

### Creazione tensori e dataloader
Partendo dagli array numpy, contenenti le immagini, dividiamo l'input e l'output per il training e il test set, che successivamente utilizziamo per creade il `TensorDataset`.

Tramite il `TensorDataset`, creiamo i `DataLoader`: utilizziamo un batch da 32, droppando l'ultimo batch per evitare incogruenze durante il training.
Impostiamo anche il parametro `shuffle` a `True`: non ha importanza l'ordine con cui i dati vengono forniti al modello. È anche necessario visto che i nuovi dati generati, concatenati al dataset, sono stati inseriti in ordine di "emozione".

In [ ]:
train_x, train_y = torch.tensor(train_set.pixels), torch.tensor(train_set.emotion)
test_x, test_y = torch.tensor(test_set.pixels), torch.tensor(test_set.emotion)

train_dataset = torch.utils.data.TensorDataset(train_x, train_y)
test_dataset = torch.utils.data.TensorDataset(test_x, test_y)

train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, drop_last=True, batch_size=32)
test_loader = torch.utils.data.DataLoader(test_dataset, shuffle=True, drop_last=True, batch_size=32)

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

### Mostrare le immagini
Definiamo la seguente funzione per visualizzare le immagini:

In [ ]:
def show(img):
    img = img * 255
    plt.axis("off")
    plt.imshow(img, cmap = 'gray')
    plt.show()

show(fer2013.pixels[4][0])

# Modello con CNN

### Definizione del modello

#### 3 Conv2D + 2 Linear

In [ ]:
class Emotion_CNN(torch.nn.Module):
    def __init__(self):
        super(Emotion_CNN, self).__init__()
    
        self.num_input_conv = 1
        self.num_filter_conv1 = 8
        self.num_filter_conv2 = 16
        self.num_filter_conv3 = 32

        self.cnn_layers = torch.nn.Sequential(
            # Primo convolution layer
            torch.nn.Conv2d(self.num_input_conv, self.num_filter_conv1, kernel_size=3, stride=1, padding=1),      
            torch.nn.BatchNorm2d(self.num_filter_conv1),
            torch.nn.MaxPool2d(kernel_size=3, stride=1),
            torch.nn.ReLU(),
            # Secondo 2D convolution layer
            torch.nn.Conv2d(self.num_filter_conv1, self.num_filter_conv2, kernel_size=3, stride=1, padding=0),
            torch.nn.BatchNorm2d(self.num_filter_conv2),
            torch.nn.MaxPool2d(kernel_size=3, stride=1),
            torch.nn.ReLU(),
            # Terzo another 2D convolution layer
            torch.nn.Conv2d(self.num_filter_conv2, self.num_filter_conv3, kernel_size=3, stride=1, padding=0),
            torch.nn.BatchNorm2d(self.num_filter_conv3),
            torch.nn.MaxPool2d(kernel_size=6, stride=2),
            torch.nn.ReLU(),
        )
 
        self.linear_layers = torch.nn.Sequential(
            torch.nn.Linear(10368, 1024),
            torch.nn.ReLU(),
            torch.nn.Linear(1024, 7)
        )
 
    # Funzione di forward
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

#### 2 Conv2D + 1 Linear

In [ ]:
class Emotion_CNN(torch.nn.Module):
    def __init__(self):
        super(Emotion_CNN, self).__init__()
    
        self.num_imput_conv = 1
        self.num_filter_conv1 = 16
        self.num_filter_conv2 = 32

        self.cnn_layers = torch.nn.Sequential(
            # Primo convolution layer
            torch.nn.Conv2d(self.num_imput_conv, self.num_filter_conv1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(self.num_filter_conv1),
            torch.nn.MaxPool2d(kernel_size=3, stride=1),
            # Secondo 2D convolution layer
            torch.nn.Conv2d(self.num_filter_conv1, self.num_filter_conv2, kernel_size=3, stride=1, padding=0),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(self.num_filter_conv2),
            torch.nn.MaxPool2d(kernel_size=3, stride=2),
            torch.nn.ReLU(),
        )
 
        self.linear_layers = torch.nn.Sequential(
            torch.nn.Linear(self.num_filter_conv2 * (21 * 21), 7),
            torch.nn.Sigmoid()
        )
 
    # Funzione di forward    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

### Istanza del modello

Nel blocco successivo impostiamo il device per il modello, in base alle disponibilità della macchina.
Per l'optimizer abbiamo scelto `Adam`: il `learning rate` migliore, nel nostro caso, sembra essere 0.00001.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Emotion_CNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()
device

### Definizione di Training e Validation

In [ ]:
train_losses = []
valid_losses = []

def Train():
    running_loss = .0
    model.train()

    for inputs,labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        
        preds = model(inputs.float())
        loss = criterion(preds,labels)
        loss.backward()
        optimizer.step()
        running_loss += loss

    train_loss = running_loss/len(train_loader)
    train_losses.append(train_loss.detach().cpu().data.numpy())
    
    print(f'train_loss {train_loss}')
    
def Valid():
    running_loss = .0
    
    model.eval()
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            preds = model(inputs.float())
            loss = criterion(preds,labels)
            running_loss += loss
            
        valid_loss = running_loss/len(test_loader)
        valid_losses.append(valid_loss.detach().cpu().data.numpy())
        print(f'valid_loss {valid_loss}')

### Allenamento e Validazione del modello

In [ ]:
epochs = 20

for epoch in range(epochs):
    print('epochs {}/{}'.format(epoch+1,epochs))
    Train()
    Valid()

### Visualizzazione della loss
In questo grafico confrontiamo la variazione dei valori della loss durante le epoche. In molti dei nostri test la curva associata ai valori di loss calcolati durante la validazione superava quella di quelli di train, indicando così un fenomeno di **overfitting**.

In [ ]:
loss_df = pd.DataFrame( {'Train Losses':train_losses, 'Valid Losses':valid_losses }, index=list(range(len(train_losses))) )

loss_df=loss_df.astype(float)

xs = loss_df.plot()
_ = xs.set_ylabel("loss")
_ = xs.set_xlabel("epoch")

### Salvataggio dello stato di modello
Per evitare di dover allenare nuovamente un certo modello, abbiamo deciso di salvare il suo stato, in modo tale da poterlo ricaricare successivamente quando necessario.

In [ ]:
if IN_COLAB:
    torch.save(model.state_dict(), '/content/drive/My Drive/Università/Deep Learning/emotion-cnn/files/model_2021-12-08.pth')
else:
    torch.save(model.state_dict(), 'files/modello_ottimo_54acc.pth')

### Caricamento dello stato di modello

In [ ]:
# Attenzione: "model" deve essere della stessa classe dell'oggetto che si aveva salvato
model.load_state_dict(torch.load('files/modello_ottimo.pth', map_location=torch.device('cuda')))

# Predizione e valutazione del modello

### Valutazione del modello
Verifichiamo l'accuratezza del modello generale e per classe tramite il successivo blocco di codice.

Salviamo anche i risultati del modello e le label perché ci serviranno per il calcolo della matrice di confusione.

In [ ]:
# Variabili per il calcolo dell'accuratezza generale
correct = 0
total = 0

# Liste necessarie per il calcolo della matrice di confusione
y = []
y_pred = []

# Calcolo dell'accuratezza del modello (necessario per dividere per classe)
class_correct = list(0. for _ in range(7))
class_total = list(0. for _ in range(7))

with torch.no_grad():
    for data in test_loader:
        images, labels = data 
        images, labels = images.cuda(), labels.cuda()
        outputs = model(images)
        _, predicted = torch.max(outputs, 1) # Supponiamo che il valore più altro sia l'emozione trovata

        y.extend(labels.squeeze().detach().cpu().numpy())
        y_pred.extend(predicted.squeeze().detach().cpu().numpy())

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        c = (predicted == labels).squeeze()
        for i in range(8):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


print('Accuratezza del modello: %d %%' % (100 * correct / total))

for i in range(7):
    print('Accuratezza %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))

### Matrice di confusione
La matrice di confusione viene calcolata tramite gli output del modello e le label reali tramite la funzione `sklearn.metrics.confusion_matrix`.
Per la visualizzazione utilizziamo a libreria `seaborn` e dividiamo ogni riga per il suo totale, in modo da ottenere i valori percentuali.

In [ ]:
cf_matrix = confusion_matrix(y, y_pred)
df_cm = pd.DataFrame(cf_matrix/np.sum(cf_matrix, axis=1), index = [i for i in classes], columns = [i for i in classes])
plt.figure(figsize = (12,7))
sn.heatmap(df_cm, annot=True)

### Altre metriche di valutazione
`classification_report` di `sklearn` permette di ottenere un riassunto, per classe, delle metriche di _precision_, _recall_, _f1-score_ e _accuratezza_.
Le aggregazioni di questi valori vengono effettuate tramite media artimetica e media pesata.

La _precision_ è il rapporto tra le volte in cui una certa classe è stata riconosciuta correttamente e tutte le volte che è stata predetta.

La _recall_ è il rapporto tra il numero di volte in cui una certa label è stata predetta correttamente e il numero totale di label di quel tipo. 

L'_f1-score_ è la media armonica tra la precision e la recall. Viene utilizzata proprio la media armonica perché viene ritenuta appropiata per fare medie tra tassi.

In [ ]:
print(classification_report(y, y_pred, digits=7))
metrics = classification_report(y, y_pred, digits=7, output_dict=True)

print("Accuratezza generale del modello:", f"{round(metrics['accuracy'], 3)}%")
print("Precisione media del modello:", f"{round(metrics['weighted avg']['precision'], 3)}%")
print("Recall media del modello:", f"{round(metrics['weighted avg']['recall'], 3)}%")
print("F1-Score medio del modello:", f"{round(metrics['weighted avg']['f1-score'], 3)}%")

## Visualizzazione dei filtri

I blocchi di codice sottostanti mirano alla visualizzazione dei vari kernel che vanno a crearsi durante il training del modello.

Salviamo in una lista tutti i layer convoluzionali, da cui estrarremmo i kernel e a cui passeremo le immagini per vedere come vengono trasformate.

In [ ]:
model_weights = [] 
conv_layers = []

model_children = list(model.children()) # Con children otteniamo tutti i vari layer del modello
counter = 0 
# append all the conv layers and their respective weights to the list
for sequence in model_children:
    if type(sequence) == torch.nn.Sequential: 
        for layer in sequence: # Iteriamo sui layer all'interno del sequenziale
            if type(layer) == torch.nn.Conv2d:
                counter += 1
                model_weights.append(layer.weight)
                conv_layers.append(layer)

print(f"Layer convoluzionali: {counter}")

In [ ]:
plt.figure(figsize=(20, 17))
for weight in model_weights:
    for i, filter in enumerate(weight):
        plt.subplot(8, 8, i+1)
        plt.imshow(filter[0, :, :].cpu().detach(), cmap='gray')
        plt.axis('off')
        plt.savefig(f'imgs_filter/48/{i}_cnn.png')
    plt.show()

L'immagine che osserveremo attraverso i vari filtri è la seguente:

In [ ]:
show(test_set.pixels[0][0])

Ad ogni layer convoluzionale della rete passiamo l'immagine elaborata dal layer precedente, salvando il risultato in una lista

In [ ]:
# Primo passo (primo layer)
outputs = [conv_layers[0](torch.tensor(test_set.pixels)[:1].cuda())]
for i in range(1, len(conv_layers)):
    # Utilizza il risultato precedente per il nuovo layer
    outputs.append(conv_layers[i](outputs[-1]))

In [ ]:
for num_layer in range(len(outputs)):
    plt.figure(figsize=(48, 48))
    layer_viz = outputs[num_layer][0, :, :, :]
    layer_viz = layer_viz.data
    print(layer_viz.size())
    for i, filter in enumerate(layer_viz):
        plt.subplot(8, 8, i + 1)
        plt.imshow(filter.cpu(), cmap='gray') # Passiamo il risultato in cpu() per permettere la visualizzazione
        plt.axis("off")

    print(f"Salvataggio delle immagini del layer {num_layer}...")
    plt.savefig(f"./imgs_filter/48/layer_{num_layer}.png")
    plt.close()

# Bibiliografia

[Build an Image Classification Model using Convolutional Neural Networks in PyTorch](https://www.analyticsvidhya.com/blog/2019/10/building-image-classification-models-cnn-pytorch/)

[Convolutional Neural Networks for Facial Expression Recognition, Shima Alizadeh and Azar Fazel, 2017](https://arxiv.org/abs/1704.06756)

[PyTorch's NN](https://pytorch.org/docs/stable/nn.html)

[Precision and recall](https://en.wikipedia.org/wiki/Precision_and_recall)

[F-Score](https://en.wikipedia.org/wiki/F-score)

[Visualizing Filters and Feature Maps in Convolutional Neural Networks using PyTorch](https://debuggercafe.com/visualizing-filters-and-feature-maps-in-convolutional-neural-networks-using-pytorch/)
